In [ ]:
import sparseSpACE
from sparseSpACE.GridOperation import Integration
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import SpatiallyAdaptiveSingleDimensions2
%matplotlib inline

from sparseSpACE.PerformTestCase import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# evaluation points for error estimation
def evaluation_points(a, b, d):
    grid = np.meshgrid(*(np.linspace(a[i], b[i], 10) for i in range(len(a))))
    points = list(zip(*(x.flat for x in grid)))

    return points


# shortcut for romberg SASD
def get_single_dim_romberg(a, b, f, reference_solution, sasd_version,
                           slice_grouping, slice_version, container_version,
                           force_balanced_refinement_tree, grid_version = GridVersion.DEFAULT):
    grid_romberg = GlobalRombergGrid(a=a, b=b, modified_basis=False, boundary=True,
                                     grid_version=grid_version,
                                     container_version=container_version,
                                     slice_grouping=slice_grouping,
                                     slice_version=slice_version)

    operation_romberg = Integration(f=f, grid=grid_romberg, dim=dim, reference_solution=reference_solution)

    return SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_romberg, rebalancing=False, version=sasd_version,
                                              force_balanced_refinement_tree=force_balanced_refinement_tree)

# Settings
dim = 5
a = np.zeros(dim)
b = np.ones(dim)
max_tol = 20  # set to max, for better comparison
max_evaluations = 10 ** 5
evaluation_points = evaluation_points(a, b, dim)
functions = []
sasd_version = 6

# --- Smooth functions

# GenzCornerPeak: Coefficients from SGA Split Extend Paper, p. 18
coeffs = np.array([np.float64(4 * i) for i in range(1, dim + 1)])
f_genz_corner = GenzCornerPeak(coeffs=coeffs)

# GenzProductPeak: Coefficients from SGA Split Extend Paper, p. 18
coeffs = np.array([np.float64(4 * i) for i in range(1, dim + 1)])
midpoint = np.ones(dim) * 0.99
f_genz_product = GenzProductPeak(coeffs, midpoint)

# GenzContinious: Coefficients from SGA Split Extend Paper, p. 18
coeffs = np.array([np.float64(4 * i) for i in range(1, dim + 1)])
midpoint = np.ones(dim) * 0.5
f_genz_cont = GenzC0(coeffs, midpoint)

# GenzGaussian: Coefficients should be i!!
coeffs = np.array([np.float64(i) for i in range(1, dim + 1)])
midpoint = np.ones(dim) * 0.99
f_genz_gaussian = GenzGaussian(coeffs, midpoint)

# FunctionExpVar: See SGA Split Extend Paper, p. 18
f_exp_var = FunctionExpVar()

# GenzOszillatory: https://www.sfu.ca/~ssurjano/oscil.html
coeffs = np.array([np.float64(i) for i in range(1, dim + 1)])
offset = 0.5
f_genz_osz = GenzOszillatory(coeffs, offset)

# --- Discontinious functions

# GenzDiscontinious: Coefficients from SGA Split Extend Paper, p. 18
border = np.ones(dim) * 0.2
coeffs = np.array([np.float64(4 * i) for i in range(1, dim + 1)])
f_genz_disc = GenzDiscontinious(border=border,coeffs=coeffs)

# Manually override functions array:
functions = [
    f_genz_corner,
    # f_genz_product,
    # f_genz_cont,
    # f_genz_gaussian,
    # f_exp_var,
    # f_genz_osz,
    # f_genz_disc
]

# Iterate over all functions
for f in functions:
    # Plot
    f.plot(np.ones(dim)*a,np.ones(dim)*b,
           # filename=str(f.__class__.__name__)
           )
    reference_solution = f.getAnalyticSolutionIntegral(a,b)
    
    # Trapezoidal Grid
    grid_trapezoidal = GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)
    operation_trapezoidal = Integration(f=f, grid=grid_trapezoidal, dim=dim, reference_solution=reference_solution)
    trapezoidal = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_trapezoidal, rebalancing=False,
                                                     version=sasd_version,
                                                     force_balanced_refinement_tree=False)
    
    operation_trapezoidal = Integration(f=f, grid=grid_trapezoidal, dim=dim, reference_solution=reference_solution)
    trapezoidal_rebalancing = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_trapezoidal, rebalancing=True,
                                                                 version=sasd_version,
                                                                 force_balanced_refinement_tree=False)
    
    # HighOrder Grid
    grid_high_order = GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)
    operation_high_order = Integration(f=f, grid=grid_high_order, dim=dim, reference_solution=reference_solution)
    high_order = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_high_order, rebalancing=False,
                                                    version=sasd_version,
                                                    force_balanced_refinement_tree=False)
    
    # HighOrder Grid
    grid_simpson = GlobalSimpsonGrid(a=a, b=b, modified_basis=False, boundary=True)
    operation_simpson = Integration(f=f, grid=grid_simpson, dim=dim, reference_solution=reference_solution)
    simpson_full = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_simpson, rebalancing=False,
                                                      version=sasd_version,
                                                      force_balanced_refinement_tree=True)
    
    # grids for standard combi
    standard_combi_grids = [
        TrapezoidalGrid(a, b, boundary=True),
        GaussLegendreGrid(a=a, b=b)
    ]
    standard_combi_grid_names = [
        "Trapezoidal Grid",
        "Gauss-Legendre Grid"
    ]

    # Unit Slices
    r_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                    sasd_version=sasd_version,
                                    slice_grouping=SliceGrouping.UNIT,
                                    slice_version=SliceVersion.ROMBERG_DEFAULT,
                                    container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                    force_balanced_refinement_tree=False)

    r_full_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                         sasd_version=sasd_version,
                                         slice_grouping=SliceGrouping.UNIT,
                                         slice_version=SliceVersion.ROMBERG_DEFAULT,
                                         container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                         force_balanced_refinement_tree=True)

    # Slice Grouping
    r_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                       sasd_version=sasd_version,
                                       slice_grouping=SliceGrouping.GROUPED,
                                       slice_version=SliceVersion.ROMBERG_DEFAULT,
                                       container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                       force_balanced_refinement_tree=False)

    r_full_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                            sasd_version=sasd_version,
                                            slice_grouping=SliceGrouping.GROUPED,
                                            slice_version=SliceVersion.ROMBERG_DEFAULT,
                                            container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                            force_balanced_refinement_tree=True)

    # Optimized slice grouping
    r_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                                 sasd_version=sasd_version,
                                                 slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                 slice_version=SliceVersion.ROMBERG_DEFAULT,
                                                 container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                 force_balanced_refinement_tree=False)

    r_full_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                                      sasd_version=sasd_version,
                                                      slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                      slice_version=SliceVersion.ROMBERG_DEFAULT,
                                                      container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                      force_balanced_refinement_tree=True)
    
    # Interpolating Romberg
    r_interpolating_old = get_single_dim_romberg(a, b, f, reference_solution,
                                             sasd_version=sasd_version,
                                             slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                             slice_version=SliceVersion.ROMBERG_DEFAULT, # Makes no difference
                                             container_version=SliceContainerVersion.LAGRANGE_ROMBERG_OLD,
                                             force_balanced_refinement_tree=False,
                                             grid_version=GridVersion.INTERPOLATE_SUB_GRIDS)    # Interpolating Romberg

    r_interpolating = get_single_dim_romberg(a, b, f, reference_solution,
                                             sasd_version=sasd_version,
                                             slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                             slice_version=SliceVersion.ROMBERG_DEFAULT, # Makes no difference
                                             container_version=SliceContainerVersion.LAGRANGE_ROMBERG,
                                             force_balanced_refinement_tree=False,
                                             grid_version=GridVersion.INTERPOLATE_SUB_GRIDS)
    # Interpolating Full grid
    r_interpolating_full_grid = get_single_dim_romberg(a, b, f, reference_solution,
                                                       sasd_version=sasd_version,
                                                       slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                       slice_version=SliceVersion.ROMBERG_DEFAULT, # Makes no difference
                                                       container_version=SliceContainerVersion.LAGRANGE_ROMBERG,
                                                       force_balanced_refinement_tree=False,
                                                       grid_version=GridVersion.INTERPOLATE_FULL_GRID)
    
    # Interpolating Bspline
    r_interpolating_bspline = get_single_dim_romberg(a, b, f, reference_solution,
                                                     sasd_version=sasd_version,
                                                     slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                     slice_version=SliceVersion.ROMBERG_DEFAULT, # Makes no difference
                                                     container_version=SliceContainerVersion.BSPLINE_ROMBERG,
                                                     force_balanced_refinement_tree=False,
                                                     grid_version=GridVersion.INTERPOLATE_SUB_GRIDS)
    
    # Interpolating Hierarchical Langrange
    r_interpolating_hierarchical_lagrange = get_single_dim_romberg(a, b, f, reference_solution,
                                                                   sasd_version=sasd_version,
                                                                   slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                                   slice_version=SliceVersion.ROMBERG_DEFAULT, # Makes no difference
                                                                   container_version=SliceContainerVersion.HIERARCHICAL_LAGRANGE_ROMBERG,
                                                                   force_balanced_refinement_tree=False,
                                                                   grid_version=GridVersion.INTERPOLATE_SUB_GRIDS)
    
    # Romberg Constant Subtraction
    r_constant_subtraction = get_single_dim_romberg(a, b, f, reference_solution,
                                             sasd_version=sasd_version,
                                             slice_grouping=SliceGrouping.UNIT,
                                             slice_version=SliceVersion.ROMBERG_DEFAULT_CONST_SUBTRACTION,
                                             container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                             force_balanced_refinement_tree=False)

    # Simpson
    r_simson_full_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                                             sasd_version=sasd_version,
                                                             slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                             slice_version=SliceVersion.ROMBERG_DEFAULT,
                                                             container_version=SliceContainerVersion.SIMPSON_ROMBERG,
                                                             force_balanced_refinement_tree=True)

    # Trapzoidal
    r_grouped_optimized_trapezoidal_balanced = get_single_dim_romberg(a, b, f, reference_solution,
                                                                      sasd_version=sasd_version,
                                                                      slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                                      slice_version=SliceVersion.TRAPEZOID,
                                                                      container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                                      force_balanced_refinement_tree=True)

    # Balance Extrapolation grid
    grid_balanced_romberg = GlobalBalancedRombergGrid(a=a, b=b, modified_basis=False, boundary=False)

    operation_balanced_romberg = Integration(f=f, grid=grid_balanced_romberg, dim=dim, reference_solution=reference_solution)

    r_balanced =  SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_balanced_romberg, rebalancing=False,
                                                     grid_surplusses=grid_balanced_romberg, # Important!
                                                     version=sasd_version,
                                                     force_balanced_refinement_tree=True)

    # Variants that are compared
    errorOperator = ErrorCalculatorSingleDimVolumeGuided()

    algorithmArray = [
        # (trapezoidal, 1, 2, errorOperator, 'Trapezoidal Grid'),
        # (trapezoidal_rebalancing, 1, 2, errorOperator, 'Trapezoidal Grid (Rebalancing)'),
        # 
        # (high_order, 1, 2, errorOperator, 'HighOrder Grid'),
        # (simpson_full, 1, 2, errorOperator, 'Simpson Grid (Balanced)'),
        # 
        # (r_unit, 1, 2, errorOperator, 'Extrapolation Grid (Unit, Romberg, Default Romberg)'),
        # (r_full_unit, 1, 2, errorOperator, 'Extrapolation Grid (Unit, Romberg, Default Romberg, Balanced)'),
        # 
        # (r_grouped, 1, 2, errorOperator, 'Extrapolation Grid (Grouped, Romberg, Default Romberg)'),
        # (r_full_grouped, 1, 2, errorOperator, 'Extrapolation Grid (Grouped, Romberg, Default Romberg, Balanced)'),
        # 
        # (r_grouped_optimized_trapezoidal_balanced, 1, 2, errorOperator, 'Extrapolation Grid (Optimized, Trapezoid, Romberg, Balanced)'),
        # 
        # (r_grouped_optimized, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Default Romberg)'),
        # (r_full_grouped_optimized, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Default Romberg, Balanced)'),
        # 
        (r_interpolating_old, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Lagrange Romberg Old)'),
        # (r_interpolating_full_grid, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Lagrange Full Romberg)'),

        (r_interpolating, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Lagrange Romberg)'),
        (r_interpolating_bspline, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Bspline Romberg)'),
        (r_interpolating_hierarchical_lagrange, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Hierarchical Lagrange Romberg)'),

        # 
        # (r_simson_full_grouped_optimized, 1, 2, errorOperator, 'Extrapolation Grid (Grouped Optimized, Romberg, Simpson Romberg, Balanced)'),

        # (r_constant_subtraction, 1, 2, errorOperator, 'Extrapolation Grid (Unit, DefaultConstantSubtraction)'), 
        
        # (r_balanced, 1, 2, errorOperator, 'Balanced Extrapolation Grid'),
    ]
    
    function_name = f.__class__.__name__
    filename = "error_comparison_{}_{}d".format(function_name, dim)

    performTestcaseArbitraryDim(f, a, b, algorithmArray,
                                max_tol, dim, 6, grids=standard_combi_grids, evaluation_points=None,
                                max_evaluations=max_evaluations,
                                calc_standard_schemes=True, # enable for standard schemes
                                minLmin=1,
                                maxLmin=3,
                                grid_names=standard_combi_grid_names,
                                legend_title="{} function".format(function_name),
                                filename=filename,
                                save_csv=True,
                                save_plot=True
                                )